## Leer los datos limpios

In [1]:
# Importar librerías
import numpy as np
import pandas as pd 
import os

In [2]:
# Saber la ruta de trabajo
os.getcwd()

'/home/leonardo'

In [3]:
#help(pd.read_csv)

In [30]:
# importar datos
#/home/leonardo/Documentos/ITAM/MNO/Final-Project-MNO-2020/data/datos_limpios.csv

df = pd.read_csv('/home/leonardo/Documentos/ITAM/MNO/Final-Project-MNO-2020/data/datos_limpios.csv', nrows = 20000)
#df = pd.read_csv('/home/leonardo/Documentos/ITAM/MNO/Final-Project-MNO-2020/data/datos_limpios.csv')

In [31]:
# Estructura de los datos
df.shape

(560626, 272)

In [36]:
# Conservar variables de la sección 3
seccion_3 = df.loc[:, ['p10', 'p11', 'p12', 'p13', 'p14']]
seccion_3.head()

,p10,p11,p12,p13,p14
0,2.0,1.0,1.0,2.0,2.0
1,4.0,3.0,3.0,5.0,6.0
2,4.0,1.0,2.0,5.0,5.0
3,2.0,1.0,1.0,3.0,3.0
4,2.0,2.0,3.0,3.0,4.0


In [37]:
seccion_3.shape

(560626, 5)

## Hacer PCA por usando la SVD (utilizando la SVD de numpy)

Tenemos varias opcones para cargar las funciones, importarlas, mandarlras a ejecutar o ponerlas en el notebook. En este ejemplo de uso las pondré en el notebook

In [51]:
# Función para PCA
def PCA_from_SVD(A):
    """
    Función para PCA a partir de la SVD de numpy 
    params: A			matriz de datos
            num_componentes 	número de componentes deseados

    return: valores_singulares	Los valores singulares de la descomposición SVD
	    componentes		Los coeficientes para calcular los componentes principales
	    Z			Los datos transformados (componentes principales)
	    varianza_explicada	La varianza explicada por cada componente principal
    """
    
    # Centrar los datos
    A = np.array(A) # convertir los datos a un numpy array por si vienen de un DataFrame
    A_centered = A - A.mean(axis=0)
    
    # Calcular SVD
    U, S, Vt = np.linalg.svd(A_centered, full_matrices=False)
    
    # Los valores singulares
    valores_singulares = S
    
    # Los componentes (coeficientes)
    componentes = ((Vt))
    
    # Los datos transformados (componentes principales)
    Z = A_centered@np.transpose(Vt)
    
    # La varianza explicada
    varianza_explicada = S**2/np.sum(S**2)
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 60%
    n = A.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.6
    num_componentes = conteo.sum() + 1
    
    # regresar 4 objetos
    return valores_singulares[:num_componentes], componentes[:num_componentes].T, Z[:,:num_componentes], varianza_explicada[:num_componentes]


In [52]:
# Ejecutar función

valores_singulares, coeficientes, Z, varianza_explicada = PCA_from_SVD(seccion_3)

In [53]:
valores_singulares

array([1676.16876321,  962.93809139])

In [54]:
coeficientes

array([[ 0.07886797, -0.21288715],
       [ 0.26246035,  0.50632358],
       [ 0.42552634,  0.68714901],
       [ 0.60025777, -0.29921326],
       [ 0.61928374, -0.36961216]])

In [55]:
# Z es nuestro nuevo data frame de datos transformados 
Z

array([[-4.11825924,  0.79580987],
       [ 1.69335835,  0.38089233],
       [ 0.12362758, -0.94929168],
       ...,
       [ 2.22491318, -1.48140569],
       [-2.12169804, -0.66840201],
       [-0.38076673, -1.42355355]])

In [77]:
varianza_explicada

array([0.58537056, 0.19149072])

## Hacer PCA usando la SVD (utilizando la SVD obtenida del algoritmo de aproxomacion de Jacobi) 

primero necesitamos la función del algoritmo de jacobi y después la función que hace PCA usando la proximación de la SVD

In [60]:
# Función que verifica si dos vectores son ortogonales de acuerdo a cierto nivel de tolerancia
def verificar_ortogonalidad(u,v,tol = 10**-8):
    if np.linalg.norm(u) <= tol or np.linalg.norm(v) <= tol:
        resultado = 1
    elif ((np.dot(u,v))/(np.linalg.norm(u)*np.linalg.norm(v))) <= tol:
        resultado = 1
    else:
        resultado = 0
    return resultado

# Función signo
def signo(x):
    if x<0:
        sig = -1
    else:
        sig = 1
    return sig



def svd_jacobi_aprox(A,TOL,maxsweep):
    # Calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representa a la matriz U(nxm),el segundo a la matriz S(mxm) matriz diagonal
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    # Nota: Esta función estima la SVD thin,la cual calcula unicamente las m columnas de U correspondientes a los m renglones de V. De esta
    # manera las columnas restantes de U no son calculadas, provocando una mejora significativa en velocidad de ejecución comparada con la 
    # la Full SVD. Referencia: https://en.wikipedia.org/wiki/Singular_value_decomposition#Thin_SVD.
    
    #dimensiones de A
    n = A.shape[1] #numero de columnas
    m = A.shape[0] #numero de filas
    nmax =n*(n-1)/2
    
    #inicializa valores del ciclo
    ak = A
    vk = np.identity(n, dtype = float) 
    sig = ''
    uk = ak
    num_col_ortogonal = 0
    k = 0
    stop = False
        
    
    while(k<=maxsweep & num_col_ortogonal<nmax):
        num_col_ortogonal =0
        for i in range(n-1):
            for j in range(i+1,n):
                col_j = ak[:,j]
                col_i = ak[:,i]

                #comprueba ortogonalidad  
                if(verificar_ortogonalidad(col_i,col_j,TOL)==1):
                    num_col_ortogonal =num_col_ortogonal+1
                else:
                    #calcula coeficientes de la matriz
                    a = np.dot(col_i,col_i)
                    b = np.dot(col_j,col_j)
                    c = np.dot(col_i,col_j)

                    #si c es cercano a cero no actualiza
                    if(c<TOL):
                        stop =True
                        break

                    #calcula la rotacion givens que diagonaliza
                    epsilon = (b-a)/(2*c)
                    t = signo(epsilon)/(abs(epsilon)+np.sqrt(1+epsilon**2))
                    cs = 1/np.sqrt(1+t**2)
                    sn = cs*t

                    #actualiza las columnas de la matriz ak
                    temp1 = ak[:,i].copy()
                    ak[:,i] = cs*temp1-sn*ak[:,j]
                    ak[:,j] = sn*temp1+cs*ak[:,j]


                    #actualiza las columnas de la matriz vk
                    temp2 = vk[:,i].copy() 
                    vk[:,i] = cs*temp2-sn*vk[:,j]
                    vk[:,j] = sn*temp2+cs*vk[:,j]             
                #cierra else
            #cierra for j
            if(stop):
                stop = False
                break
            
        #cierra for i
        k = k+1
     #cierra while
    
      
    #Obtener sigma (normas euclidianas de columnas de ak)
    sig = np.linalg.norm(ak, axis=0)

    #Obtener U (columnas normalizadas de ak)
    for i in range(n):
        if (sig[i]<TOL):
            uk[:,i] = 0  
        else:
            uk[:,i] = ak[:,i]/sig[i]
        

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    #index <- order(sig,decreasing = TRUE)
    index = np.argsort(sig) # Obtenemos los indices de sig ordenado
    index = index[::-1] # Invertimos, para obtener los índices del orden decreciente
    #Reordenamos
    V = vk[:,index]
    s = sig[index]
    U = uk[:,index]
    
    return U, s, V  

In [61]:
def PCA_from_SVD_jacobi(A):
    """
    Función para PCA a partir de la SVD 
    params: A			matriz de datos
            num_componentes 	número de componentes deseados
    return: valores_singulares	Los valores singulares de la descomposición SVD
	    componentes		Los coeficientes para calcular los componentes principales
	    Z			Los datos transformados (componentes principales)
	    varianza_explicada	La varianza explicada por cada componente principal
    """
    
    # Centrar los datos
    A = np.array(A) # convertir los datos a un numpy array por si vienen de un DataFrame
    A_centered = A - A.mean(axis=0)
    
    # Modificar esta línea de código, mandar a llamar la función creada por el equipo 
    # Calcular SVD
    U, S, Vt = svd_jacobi_aprox(A_centered,1e-12,500)
    
    # Los valores singulares
    valores_singulares = S
    
    # Los componentes (coeficientes)
    componentes = ((Vt))
    
    # Los datos transformados (componentes principales)
    Z = A_centered@np.transpose(Vt)
    
    # La varianza explicada
    varianza_explicada = S**2/np.sum(S**2)
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 60%
    n = A.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.6
    num_componentes = conteo.sum() + 1    
    
    # regresar 4 objetos
    return valores_singulares[:(num_componentes)], componentes[:(num_componentes)].T, Z[:,:(num_componentes)], varianza_explicada[:(num_componentes)]



In [62]:
valores_singulares2, coeficientes2, Z2, varianza_explicada2 = PCA_from_SVD_jacobi(seccion_3)

In [63]:
valores_singulares2

array([1666.64673086,  964.84750317])

In [64]:
coeficientes2

array([[ 0.12360526,  0.32424786],
       [ 0.02100815, -0.44561778],
       [ 0.99138424,  0.        ],
       [-0.0369452 ,  0.83434301],
       [-0.00852871, -0.01264387]])

In [89]:
Z2

array([[-0.01442399, -0.00843885],
       [ 0.00552084, -0.00231051],
       [-0.00063262,  0.00778347],
       ...,
       [-0.00267452, -0.0015174 ],
       [ 0.01033461, -0.00687196],
       [ 0.0073078 , -0.00045593]])

In [19]:
varianza_explicada2

array([0.59370287, 0.19585792])

Con lo anterior vemos que los valores singulares y la varianza explicada son muy similares entre lo obtenido con la función de numpy y el algoritmo de Jacobi, no obstante, el en el resultado de la SVD U*S*Vh, la Vh no es muy similar a la de numpy y por ello los coeficientes de los componentes y los componentes no son similares.

## Hacer PCA usando sklearn 

In [65]:
# Importar librerías
from sklearn.decomposition import PCA

In [66]:
# Función que hace PCA
pca = PCA(n_components=2,svd_solver='full')
pca.fit(seccion_3)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [67]:
# Valores singulares
pca.singular_values_

array([1676.16876321,  962.93809139])

In [68]:
# coeficientes de los componentes
print(pca.components_.T)

[[ 0.07886797 -0.21288715]
 [ 0.26246035  0.50632358]
 [ 0.42552634  0.68714901]
 [ 0.60025777 -0.29921326]
 [ 0.61928374 -0.36961216]]


In [69]:
# Datos transformados (componentes principales)
tabla_z = pca.transform(seccion_3)
tabla_z

array([[-4.11825924,  0.79580987],
       [ 1.69335835,  0.38089233],
       [ 0.12362758, -0.94929168],
       ...,
       [ 2.22491318, -1.48140569],
       [-2.12169804, -0.66840201],
       [-0.38076673, -1.42355355]])

In [70]:
# Varianza explicada
print(pca.explained_variance_ratio_)

[0.57651235 0.1902698 ]


Como podrán notar, se obtienen los mismos valores que a partir de la primera función que calcula PCA a través de la SDV